In [1]:
from __future__ import print_function
from __future__ import division
from __future__ import unicode_literals

In [2]:
import numpy as np

## Markov Chain

<img src="img/mlst_1607.png" width="500" align='left'>

In [3]:
m_chain = np.array([[0.7, 0.2, 0.0, 0.1],
          [0.0, 0.0, 0.9, 0.1],
          [0.0, 1.0, 0., 0.],
          [0., 0., 0., 1.0]])

In [4]:
m_chain

array([[ 0.7,  0.2,  0. ,  0.1],
       [ 0. ,  0. ,  0.9,  0.1],
       [ 0. ,  1. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  1. ]])

In [5]:
curr_state = np.array([1.0, 0, 0, 0])
curr_state

array([ 1.,  0.,  0.,  0.])

In [6]:
for i in range(15):
    curr_state = np.dot(curr_state, m_chain)
curr_state

array([ 0.00474756,  0.20836289,  0.14490451,  0.64198504])

In [7]:
for i in range(1000 - 15):
    curr_state = np.dot(curr_state, m_chain)
curr_state

array([  1.25325664e-155,   4.51438816e-024,   5.80421335e-024,
         1.00000000e+000])

# Markov Decision Processes

MDP provide a mathematical framework for modeling decision making in situations where outcomes are partly random and partly under the control of a decision maker

<img src="img/mlst_1608.png" width="500" align='left'>

In [8]:
STATE_NUM = 3

nan = np.nan

T = np.array([  # shape=[s, a, s']
        [[0.7, 0.3, 0.0], [1.0, 0.0, 0.0], [0.8, 0.2, 0.0]],
        [[0.0, 1.0, 0.0], [nan, nan, nan], [0.0, 0.0, 1.0]],
        [[nan, nan, nan], [0.8, 0.1, 0.1], [nan, nan, nan]],
    ])

R = np.array([  # shape=[s, a, s']
        [[10., 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]],
        [[0.0, 0.0, 0.0], [nan, nan, nan], [0.0, 0.0, -50.]],
        [[nan, nan, nan], [40., 0.0, 0.0], [nan, nan, nan]],
    ])

possible_actions = [[0, 1, 2], [0, 2], [1]]

In [9]:
def next_action_by_random(s):
    return np.random.choice(possible_actions[s])

def next_state_by_prob(s, a):
    return np.random.choice(range(STATE_NUM), p=T[s, a])

def next_state_by_random(s, a):
    return np.random.choice(np.argwhere(T[s, a] > 0).reshape(-1))

def observe_reward(s, a, sp):
    return R[s, a, sp]

def init_Q(state_num):
    Q = np.full((state_num, state_num), -np.inf) # -inf for impossible actions
    for state, actions in enumerate(possible_actions):
        Q[state, actions] = 0.0 # Initial value = 0.0, for all possible actions
    return Q

## Value Iteration 

In [10]:
V = np.zeros(STATE_NUM)
V

array([ 0.,  0.,  0.])

In [11]:
discount_rate = 0.9

for i in range(100):
    V_prev = V.copy()
    for s in range(STATE_NUM):
        action_values = []
        for a in possible_actions[s]:
            action_v = np.sum([
                T[s, a, sp] * \
                (R[s, a, sp] + discount_rate * V_prev[sp])
                for sp in range(3)
            ])
            action_values.append(action_v)
        V[s] = np.max(action_values)
print(V)

[ 18.91891892   0.          50.13365013]


## Q Iteration

In [12]:
Q = init_Q(STATE_NUM)
Q

array([[  0.,   0.,   0.],
       [  0., -inf,   0.],
       [-inf,   0., -inf]])

![title](img/eq_139.png)

In [13]:
discount_rate = 0.95
n_iterations = 20000

In [14]:
for i in range(n_iterations):
    Q_prev = Q.copy()
    for s in range(STATE_NUM):
        for a in possible_actions[s]:
            Q[s, a] = np.sum([
                T[s, a, sp] * \
                (R[s, a, sp] + discount_rate * np.max(Q_prev[sp]))
                for sp in range(3)
            ])
print(Q)

[[ 21.89925005  20.80428755  16.86759588]
 [  1.12082922         -inf   1.17982024]
 [        -inf  53.87349498         -inf]]


In [15]:
np.argmax(Q, axis=1) # optimal action for each state

array([0, 2, 1])

# Q-Learning

Agent initially has no idea what the transition probabilities are (it does not know T(s, a, s′)), and it does not know what the rewards are going to be either (it does not know R(s, a, s′)).

![title](img/eq_142.png)

In [16]:
learning_rate0 = 0.05
learning_rate_decay = 0.1
n_iterations = 20000

In [17]:
Q = init_Q(STATE_NUM)
Q

array([[  0.,   0.,   0.],
       [  0., -inf,   0.],
       [-inf,   0., -inf]])

In [18]:
s = 0
for i in range(n_iterations):
    a = next_action_by_random(s)
    sp = next_state_by_random(s, a)
    reward = observe_reward(s, a, sp)
    learning_rate = learning_rate0 / (1 + i * learning_rate_decay)
    Q[s, a] = ((1 - learning_rate) * Q[s, a] +
                learning_rate * (reward + discount_rate*np.max(Q[sp])))
    s = sp
    
print(Q)

[[  2.13388094   0.39599773   0.21791899]
 [  0.                 -inf -23.2288997 ]
 [        -inf  10.82558957         -inf]]


In [19]:
np.argmax(Q, axis=1) # optimal action for each state

array([0, 0, 1])